In [7]:
import requests
import pprint
import json
from wikidataintegrator import wdi_core, wdi_login

# Server settings

In [46]:
mediawiki_api_url = "http://127.0.0.1:7171/w/api.php"
sparql_endpoint_url = "http://127.0.0.1:7272/proxy/wdqs/bigdata/namespace/wdq/sparql"
username = "Test"
password = "test2"
login = wdi_login.WDLogin(username, password, mediawiki_api_url=mediawiki_api_url)

http://127.0.0.1:7171/w/api.php
Successfully logged in as Test


# Create the properties

In [47]:
def createProperty(login, label, description, property_datatype, lang="en"):
  s = []
  localEntityEngine = wdi_core.WDItemEngine.wikibase_item_engine_factory(mediawiki_api_url,sparql_endpoint_url)
  item = localEntityEngine(data=s)
  item.set_label(label, lang=lang)
  item.set_description(description, lang=lang)
  print(item.write(login, entity_type="property", property_datatype=property_datatype))
    

## Item properties

createProperty(login, "category", "", "wikibase-item", lang="en")
createProperty(login, "date", "", "wikibase-item", lang="en")
createProperty(login, "facilitator", "", "wikibase-item", lang="en")
createProperty(login, "location", "", "wikibase-item", lang="en")
createProperty(login, "tag", "", "wikibase-item", lang="en")

## String properties
createProperty(login, "duration", "", "string", lang="en")
createProperty(login, "time block", "", "string", lang="en")
## URL properties
createProperty(login, "link", "", "url", lang="en")


## external-identifier
createProperty(login, "id", "identifier", "external-id", lang="en")

## date-time
createProperty(login, "start time", "", "time", lang="en")



P1
P2
P3
P4
P5
P6
P7
P8
P9
P10


In [48]:
props = dict()
results = wdi_core.WDItemEngine.execute_sparql_query("SELECT * WHERE { ?item rdf:type wikibase:Property ; rdfs:label ?label }", endpoint=sparql_endpoint_url)
for result in results["results"]["bindings"]:
    print(result["item"]["value"], result["label"]["value"])
    props[result["label"]["value"]]=result["item"]["value"].replace("http://wikibase.svc/entity/", "")
props

http://wikibase.svc/entity/P1 category
http://wikibase.svc/entity/P2 date
http://wikibase.svc/entity/P3 facilitator
http://wikibase.svc/entity/P4 location
http://wikibase.svc/entity/P5 tag
http://wikibase.svc/entity/P6 duration
http://wikibase.svc/entity/P7 time block
http://wikibase.svc/entity/P8 link
http://wikibase.svc/entity/P9 id
http://wikibase.svc/entity/P10 start time


{'category': 'P1',
 'date': 'P2',
 'facilitator': 'P3',
 'location': 'P4',
 'tag': 'P5',
 'duration': 'P6',
 'time block': 'P7',
 'link': 'P8',
 'id': 'P9',
 'start time': 'P10'}

In [50]:
with open('data.json') as f:
    data = json.load(f)

for key in data.keys():
    print(key)
#pprint.pprint(data)

for session in data["sessions"]:
    statements = []
    if len(wdi_core.WDItemEngine.get_wd_search_results(search_string=session["category"], mediawiki_api_url=mediawiki_api_url)) == 0:
        categoryPage = wdi_core.WDItemEngine(new_item=True, mediawiki_api_url=mediawiki_api_url, sparql_endpoint_url=sparql_endpoint_url)
        categoryPage.set_label(session["category"], lang="en")
        try:
            qid=categoryPage.write(login)
        except:
            continue
    statements.append(wdi_core.WDItemID(value=qid, prop_nr=props["category"]))
    
    # location
    if len(wdi_core.WDItemEngine.get_wd_search_results(search_string=session["location"], mediawiki_api_url=mediawiki_api_url)) == 0:
        categoryPage = wdi_core.WDItemEngine(new_item=True, mediawiki_api_url=mediawiki_api_url, sparql_endpoint_url=sparql_endpoint_url)
        categoryPage.set_label(session["location"], lang="en")
        try:
            qid=categoryPage.write(login)
        except:
            continue
    statements.append(wdi_core.WDItemID(value=qid, prop_nr=props["location"]))
    
    
    #day
    if session["day"] == "Friday":
        date = "" ## Add datestring TODO later
    
    #duration
    statements.append(wdi_core.WDString(value=session["duration"],prop_nr=props["duration"]))
    
    #link
    statements.append(wdi_core.WDUrl(value=session["link"], prop_nr=props["link"]))
    
    #timeblock
    #statements.append(wdi_core.WDString(value=session["timeblock"], prop_nr=props["timeblock"]))
    
    wbPage = wdi_core.WDItemEngine(new_item=True, data=statements, mediawiki_api_url=mediawiki_api_url, sparql_endpoint_url=sparql_endpoint_url)
    wbPage.set_label(session["title"], lang="en")
    if len(session["description"])>249:
        wbPage.set_description(session["description"].replace("<div>", "").replace("</div>", "")[:245]+"...", lang="en")
    else:
        wbPage.set_description(session["description"].replace("<div>", "").replace("</div>", ""), lang="en")
    print(wbPage.write(login))


    facilitator_array = session["facilitator_array"]
    facilitator = session["facilitators"]

    location = session["location"]
    start = session["start"]

    timeblock = session["timeblock"]
    tags = session["tags"]
    #print(session.keys())

sessions
timeblocks
Q3
Q5
Q7
Q8
Q10
Q12
Q13


KeyError: 'success'

In [32]:
pprint.pprint(data)

{'sessions': [{'category': 'Everyone',
               'day': 'Friday',
               'description': '<div>Opening Session, including introduction '
                              'speech and practical details about the '
                              'conference</div>',
               'duration': '25min',
               'facilitator_array': ['<a '
                                     'href="https://www.wikidata.org/wiki/User:Lea_Lacroix_(WMDE)" '
                                     'title="User:Lea Lacroix (WMDE)">Léa '
                                     'Lacroix</a>'],
               'facilitators': '<a '
                               'href="https://www.wikidata.org/wiki/User:Lea_Lacroix_(WMDE)" '
                               'title="User:Lea Lacroix (WMDE)">Léa '
                               'Lacroix</a>',
               'id': 'SE-0',
               'link': 'https://www.wikidata.org/wiki/Wikidata:WikidataCon_2019/Program/Sessions/Opening_session',
               'location': '

                              'programming will be necessary to follow the '
                              'discussion, we will use R in the course of the '
                              'session to illustrate where, what, and how can '
                              'be done with our numbers. Finally, we discuss '
                              'the possibilities to act upon the introduced '
                              'statistics and indicators to potentially '
                              'improve the existing and establish new '
                              'connections among different Wikimedia '
                              'communities.</div>',
               'duration': '25min',
               'facilitator_array': ['Goran S. Milovanović'],
               'facilitators': 'Goran S. Milovanović',
               'id': 'SUB-091',
               'link': 'https://www.wikidata.org/wiki/Wikidata:WikidataCon_2019/Program/Sessions/Wikidata_Statistics:_What,_Where,_and_How%3F',
        

In [5]:
for session in data["sessions"]:
    print(session)

{'category': 'Everyone', 'day': 'Friday', 'description': '<div>Opening Session, including introduction speech and practical details about the conference</div>', 'duration': '25min', 'facilitator_array': ['<a href="https://www.wikidata.org/wiki/User:Lea_Lacroix_(WMDE)" title="User:Lea Lacroix (WMDE)">Léa Lacroix</a>'], 'facilitators': '<a href="https://www.wikidata.org/wiki/User:Lea_Lacroix_(WMDE)" title="User:Lea Lacroix (WMDE)">Léa Lacroix</a>', 'id': 'SE-0', 'link': 'https://www.wikidata.org/wiki/Wikidata:WikidataCon_2019/Program/Sessions/Opening_session', 'location': 'Kleist', 'nextSessionID': 'SE-3', 'nextSessionTitle': 'Glimpse over Wikidata', 'start': '10:00', 'tags': '', 'timeblock': 'tb-friday', 'title': 'Opening session'}
{'category': 'Presentation', 'day': 'Friday', 'description': '<div>TBD</div>', 'duration': '25min', 'facilitator_array': ['Lydia Pintscher'], 'facilitators': 'Lydia Pintscher', 'id': 'SE-3', 'link': 'https://www.wikidata.org/wiki/Wikidata:WikidataCon_2019/Pro